### The paddle version using in this file is 1.8.4

In [ ]:
import paddle
paddle.__version__

### Define constant

In [ ]:
import numpy as np
import paddle.fluid as fluid
c = fluid.layers.fill_constant(shape=[3,3],dtype='float32',value=3.14) # Declaration
print(c)

cpu = fluid.core.CPUPlace() # load CPU
exe = fluid.Executor(cpu) # execute program using CPU
exe.run(fetch_list=[c]) # get the value of 'c'

### Define variables

#### Method I

In [ ]:
import paddle.fluid as fluid
import numpy

# 1、Define the variables (Data variables)
a1 = fluid.data(name="a1",shape=[1],dtype='float32') 
b1 = fluid.data(name="b1",shape=[1],dtype='float32') 

# 2、Define the operation
add = fluid.layers.elementwise_add(a1,b1) # add
mul = fluid.layers.elementwise_mul(a1,b1) # multiply(numbers)


# 3、Define the executor
# using CPU
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)

# 4、Assignment and operation
x1 = numpy.array([5]).astype("float32")
y1 = numpy.array([7]).astype("float32")
outs = exe.run(
        feed={'a1':x1,'b1':y1},
        fetch_list=[add,mul])
print(outs) # 5+7 = 12 5*7 = 35

#### Method II

In [ ]:
import paddle.fluid as fluid
import numpy

# 1、Define the variables (Parameter variables)
x = fluid.layers.create_parameter(shape = [1],dtype='float32',name = 'x')
y = fluid.layers.create_parameter(shape = [1],dtype='float32',name = 'y')

# 2、Define the operation
sub = fluid.layers.elementwise_sub(x,y,name = 'sub') # sub

# 3、Define the executor
# Using CPU
cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)

# 4、Assignment and operation
a = numpy.array([15]).astype("float32")
b = numpy.array([7]).astype("float32")
outs = exe.run(
        feed={'x':a,'y':b}, # feed the data
        fetch_list=[sub])

print(outs) # 15-7 = 8

### Add&Mean

In [ ]:
import numpy as np

In [ ]:
np.arange(1,101) # create array [1,101) == [1,100]

In [ ]:
from paddle import fluid
import numpy as np

x = fluid.data(name="x",shape=[1],dtype='float32')
y = fluid.data(name="y",shape=[1],dtype='float32')

result = fluid.layers.elementwise_add(x,y) # add

a = fluid.layers.range(1,101,1,'int32') # 从 same method as np.arange
b = fluid.layers.cast(a,dtype='float64') # using cast for data type conversion
m = fluid.layers.mean(b) # mean

c = np.array([255]).astype("float32")
d = np.array([3]).astype("float32")

cpu = fluid.core.CPUPlace()
exe = fluid.Executor(cpu)

exe.run(feed = {'x':c,'y':d},fetch_list=[result,a,b,m]) # 255 + 3

### Matrix Operations

In [ ]:
import paddle.fluid as fluid
import numpy as np

a = fluid.layers.data(name='xx', shape=[2, 3], dtype='float32')
b = fluid.layers.data(name='yy', shape=[3, 2], dtype='float32')
out = fluid.layers.matmul(a, b) # matrix multiply


a=np.array([[1,2,3],[3,5,7]]).astype('float32')
b=np.array([[1,9],[2,8],[3,7]]).astype('float32')

print(a,b)
display(a,b)

place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)

out_y = exe.run(program=fluid.default_main_program(),# using main_program
               feed={'xx': a, 'yy': b},
               fetch_list=[out])
out_y

### Martix dot product

In [ ]:
import paddle.fluid as fluid
import numpy as np

a = fluid.layers.data(name='xx', shape=[3], dtype='float32')
b = fluid.layers.data(name='yy', shape=[3], dtype='float32')

out = fluid.layers.dot(a, b) # dot product

a=np.array([1,2,3]).astype('float32')
b=np.array([1,9,2]).astype('float32')

place = fluid.CPUPlace()
exe = fluid.executor.Executor(place)

out_y= exe.run(feed={'xx': a, 'yy': b},
               fetch_list=[out])
out_y

### parameter obtain

In [ ]:
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr import numpy as np

x = fluid.layers.data(name='x', shape=[None,5], dtype='float32') # None == -1
y = fluid.layers.data(name='y', shape=[None,5], dtype='float32') 

# Define the parameters attribute
param_w = ParamAttr(name='target_fc') # parameter(slope)
bias_b = ParamAttr(name='target_fc_b') # bias(intercept)

# fully connected
# auto generate w and b to execute matrix operation
fc1 = fluid.layers.fc(input=x, # data 
                      size=2, # output units
                      act=None, # activation
                      param_attr=param_w, 
                      bias_attr=bias_b) 
fc2 = fluid.layers.fc(input=y, # data 
                      size=2, # output units
                      act=None, # activation
                      param_attr=param_w, 
                      bias_attr=bias_b) 

place = fluid.CPUPlace()
exe = fluid.Executor(place)

# initialization
exe.run(fluid.default_startup_program())
w = fluid.global_scope().find_var("target_fc").get_tensor()
b = fluid.global_scope().find_var("target_fc_b").get_tensor()
w = np.array(w)
b = np.array(b)

#print("the shape of w is: ",w.shape)
#print("the shape of b is: ",b.shape)
print(w)
print(b)

a = np.asarray([1,2,3,4,5]).astype(np.float32).reshape(1,5)
b = np.asarray([6,7,8,9,10]).astype(np.float32).reshape(1,5)
print(a)
print(b)

out = exe.run(feed = {'x':a, 'y':b},fetch_list = [fc1,fc2])
display(out)

In [ ]:
# Equivalent to matrix dot product
print(a.dot(w))
print(b.dot(w))

### program


In [ ]:
import numpy as np
import paddle.fluid as fluid

x = fluid.data(name='x',shape=[None, 13], dtype='float32')
y = fluid.layers.data(name='y', shape=[1], dtype='float32')

y_predict = fluid.layers.fc(input=x, size=1, act=None)
# Mean square error
cost = fluid.layers.square_error_cost(input=y_predict, label=y)

cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)

exe.run(fluid.default_startup_program())

a = np.random.randint(0,10,size =[1,13]).astype(np.float32)
b = np.random.randint(0,10,size = 1).astype(np.float32)

print(a,b)


y_,cost_ = exe.run(feed = {'x':a,'y':b},fetch_list=[y_predict,cost])
display(y_,cost_)

In [ ]:
((y_ - b)**2).mean() # the content of MSE